# Processing tweets

Now that we have scraped the tweets - will process them to get the best results.

A few basic things to do are:

* All to lower

* Remove non-alphanumeric

* Remove hashtags (potentially)

* Entity recognition (to combine names)

* Lemmatize

All throughout this should be drawing some wordclouds to see how things are shaping up and making sure that we are getting the right stuff.

In [1]:
# Libraries
import pandas as pd
import matplotlib.pyplot as plt
import spacy, wordcloud
import nltk, langdetect
from collections import Counter

# Paths
home_dir = 'c:/Users/amathewl/Desktop/La Caixa TM/Python/'
salida_dir = home_dir + 'Salida/'

# File names
input_tweets = salida_dir + 'all_tweets_ext.csv'
output_tweets = salida_dir + 'cleaned_tweets.csv'

# Get our spacy model - python -m spacy download es / or .load('es_core_news_md')
nlp = spacy.load('es_core_news_md')

## Define global functions

Define our wordcloud generator here because we will use it a lot

In [2]:
def get_wordcloud(documents, stopwords = [], width = 800, height = 400):
    '''
    Wrapper to get the word cloud - note that this AUTOMATICALLY considers bigrams
    Note that will also balance the absolute rank of a word and the number of times it appears
    
    Expecting list of documents as documents
    '''
    wc = wordcloud.WordCloud(stopwords = stopwords,
                            width = width, height = height).generate(' '.join(documents))
    
    # Make bigrams appear with an underscore for clarity
    wc.layout_ = [((layout[0][0].replace(' ', '_'), layout[0][1]), layout[1], layout[2], layout[3], layout[4]) for layout in wc.layout_]
    
    plt.figure(figsize = (200, 200))
    plt.imshow(wc)
    plt.axis('off')
    plt.show()

## Loading data

Load in the tweets from CSV and check that everything looks good

In [3]:
tweet_data = pd.read_csv(input_tweets, encoding = 'utf-8')

print('The data has shape {}'.format(tweet_data.shape))
print('It looks like this:')
print(tweet_data.head(10))

The data has shape (4691, 2)
It looks like this:
                                                Text            User
0  RT @edugaresp: 2) Perdonar 5.124 millones € al...      p_pmorales
1  RT @expansioncom: CaixaBank financiará durante...  AndyArquitecto
2  @caixabank destina 3.000 millones a la moderni...      Economia_3
3  RT @expansioncom: CaixaBank financiará durante...    AgenciaSCorp
4  RT @El_Plural: Todo lo que debes tener en cuen...        caval100
5  RT @expansioncom: CaixaBank financiará durante...  gutirrez_lvaro
6  RT @edugaresp: 2) Perdonar 5.124 millones € al...       maisaruro
7  RT @edugaresp: 2) Perdonar 5.124 millones € al...        ZCelemin
8  CaixaBank coloca 1.250 millones en CoCos con u...        d_lozano
9  RT @edugaresp: 2) Perdonar 5.124 millones € al...      maripilito


In [4]:
user_counts = Counter(tweet_data['User'].values)
user_counts.most_common(10)

[('caixabank', 158),
 ('caixabank_cat', 97),
 ('sacamoko', 31),
 ('Apedirjusticia1', 25),
 ('Gregor_Wegener', 22),
 ('raulmirsa', 21),
 ('TreceRosas3', 21),
 ('AccionistasCABK', 20),
 ('Forinvest', 19),
 ('AccionistesCABK', 19)]

In [5]:
# Drop duplicates - NOTE that if different users are tweeting it won't be removed...
tweet_data.drop_duplicates(inplace = True)

print('Have dropped duplicates - now have shape {}'.format(tweet_data.shape))

# Remove those from Caixa bank
tweet_data = tweet_data[~tweet_data['User'].isin(['caixabank', 'caixabank_cat'])]
print('Have removed caixabank - now have shape {}'.format(tweet_data.shape))

# Remove the initial part of the strings if a RT and delete duplicates again
tweet_data['Text'] = tweet_data['Text'].replace({'RT [\S]*: ' : ''}, regex = True)
tweet_data.drop_duplicates(inplace = True)

print('The data now looks like:\n{}\n'.format(tweet_data.head(10)))

print('Have removed duplicates again - now have shape {}'.format(tweet_data.shape))

Have dropped duplicates - now have shape (4661, 2)
Have removed caixabank - now have shape (4406, 2)
The data now looks like:
                                                Text            User
0  2) Perdonar 5.124 millones € al banco Santande...      p_pmorales
1  CaixaBank financiará durante dos años al secto...  AndyArquitecto
2  @caixabank destina 3.000 millones a la moderni...      Economia_3
3  CaixaBank financiará durante dos años al secto...    AgenciaSCorp
4  Todo lo que debes tener en cuenta al pedir un ...        caval100
5  CaixaBank financiará durante dos años al secto...  gutirrez_lvaro
6  2) Perdonar 5.124 millones € al banco Santande...       maisaruro
7  2) Perdonar 5.124 millones € al banco Santande...        ZCelemin
8  CaixaBank coloca 1.250 millones en CoCos con u...        d_lozano
9  2) Perdonar 5.124 millones € al banco Santande...      maripilito

Have removed duplicates again - now have shape (4402, 2)


In [6]:
# Get initial wordcloud
get_wordcloud(tweet_data['Text'].values)

# Huge amount of stop words so this wordcloud is pretty much useless

## Basic data processing

Will do the most basic processing steps so that we can get some initial word clouds

Will do this with spacy - works like "for token in doc for doc in nlp" - some things to look out for:

* token.text = word

* token.lemma_ = its lemma

* token.pos_ = part of speech

* token.shape_ = shape in form Xxxx

* token.is_alpha = if alphanumeric

* token.is_stop = if stopword

Entities is like "for ent in doc.ents for doc in nlp":

* ent.text

* ent.label_

In [7]:
# Print the first 20 results just to see what this looks like
tweet_list = tweet_data['Text'].values

for i in range(5):
    doc = nlp(tweet_list[i])
    nlp_list = [(token.lemma_, token.pos_, token.is_alpha, token.is_stop) for token in doc]
    entity_list = [(ent.text, ent.label_) for ent in doc.ents]
    
    print('The original doc is: \n{}'.format(doc))
    print('The NLP (lemma, pos, alphanumeric, stopword) details of the doc are: \n{}'.format(nlp_list))
    print('The entities of the doc are: \n{}'.format(entity_list))
    print('\n\n')

The original doc is: 
2) Perdonar 5.124 millones € al banco Santander, a CaixaBank y a Bankia en el impuesto de Sociedades gracias a los créditos…
The NLP (lemma, pos, alphanumeric, stopword) details of the doc are: 
[('2', 'NUM', False, False), (')', 'PUNCT', False, False), ('Perdonar', 'VERB', True, False), ('5.124', 'NUM', False, False), ('millón', 'NOUN', True, False), ('€', 'VERB', False, False), ('al', 'ADP', True, True), ('banco', 'NOUN', True, False), ('Santander', 'PROPN', True, False), (',', 'PUNCT', False, False), ('a', 'ADP', True, False), ('CaixaBank', 'PROPN', True, False), ('y', 'CONJ', True, False), ('a', 'ADP', True, False), ('Bankia', 'PROPN', True, False), ('en', 'ADP', True, True), ('el', 'DET', True, True), ('imponer', 'NOUN', True, False), ('de', 'ADP', True, True), ('Sociedades', 'PROPN', True, False), ('gracia', 'NOUN', True, False), ('a', 'ADP', True, False), ('lo', 'DET', True, True), ('crédito', 'NOUN', True, False), ('…', 'PUNCT', False, False)]
The entities

One of the first things that I can see that is clear - is that we should probably be removing

* Tweets starting with @CaixaBank

* Removing the RE @...: and then removing duplicates

In [8]:
def basic_treatment(tweet_list, pos_remove = [], special_words = []):
    '''
    Function that does some basic treatment of the data according to spacy stuff
    '''
    
    treated_data = []
    i = 0
    for tweet in tweet_list:
        if i % 500 == 0:
            print('Have processed {} tweets'.format(i))

        doc = nlp(tweet)

        # Maybe have to think of a way to deal with people
        token_list = [token.lemma_.lower() for token in doc if token.is_alpha and not token.is_stop and token.pos_ not in pos_remove and token.text.lower() not in special_words]
        treated_data.append(' '.join(token_list))

        i += 1

    print('Treated data - shape is {}'.format(len(treated_data)))   
    
    return treated_data

In [9]:
# POS that we will remove
pos_remove = ['ADP', 'NUM', 'CONJ']
special_words = ['ibex35']

treated_data = basic_treatment(tweet_list, pos_remove)

Have processed 0 tweets
Have processed 500 tweets
Have processed 1000 tweets
Have processed 1500 tweets
Have processed 2000 tweets
Have processed 2500 tweets
Have processed 3000 tweets
Have processed 3500 tweets
Have processed 4000 tweets
Treated data - shape is 4402


In [10]:
for i in range(10):
    print('The original tweet was: \n{}'.format(tweet_list[i]))
    print('The new tweet is: \n{}'.format(treated_data[i]))
    print('\n\n')

The original tweet was: 
2) Perdonar 5.124 millones € al banco Santander, a CaixaBank y a Bankia en el impuesto de Sociedades gracias a los créditos…
The new tweet is: 
perdonar millón banco santander caixabank bankia imponer sociedades gracia crédito



The original tweet was: 
CaixaBank financiará durante dos años al sector hostelero con 3.000 milllones de euros https://t.co/cdYO0EqxQH https://t.…
The new tweet is: 
caixabank financiar año sector hostelero milllones euro



The original tweet was: 
@caixabank destina 3.000 millones a la modernización del sector turístico y hotelero. https://t.co/G1KXqCxuZi https://t.co/PgrK7nkFtr
The new tweet is: 
destinar millón modernización sector turístico hotelero



The original tweet was: 
CaixaBank financiará durante dos años al sector hostelero con 3.000 milllones de euros https://t.co/cdYO0EqxQH https://t.…
The new tweet is: 
caixabank financiar año sector hostelero milllones euro



The original tweet was: 
Todo lo que debes tener en cuen

In [11]:
get_wordcloud(treated_data)

## Advanced treatment

Now will start doing some more advanced text specific treamtent like word replacement and stuff.

I think will probably drop that one tweet that is repeated a lot.

In [12]:
# First add as a column
tweet_data['Clean_text'] = treated_data

In [13]:
# Drop the duplicate original tweets
tweet_data.drop_duplicates(subset = 'Text', inplace = True)

print('Have dropped duplicates - now have shape {}'.format(tweet_data.shape))

Have dropped duplicates - now have shape (2498, 3)


In [14]:
# Check how many are spanish
tweet_data['Tweet_language'] = [langdetect.detect(tweet) if tweet != '' else 'es' for tweet in tweet_data['Clean_text'].values]

In [15]:
# Only remove those that are english because not sure of others
tweet_data = tweet_data[~tweet_data['Tweet_language'].isin(['en', 'ru'])]

print('Have removed english tweets - now have shape {}'.format(tweet_data.shape))

Have removed english tweets - now have shape (2388, 4)


Now will do some word replacements and removals, especially Catalan to Spanish

Will load in an Excel of two columns where the first is to be replaced and second is replace. There is definitely a better way to do the translation - but for now don't really have time to investigate.

In [16]:
def build_replacedic(excel_path):
    '''
    Give an Excel path and build a dictionary of form "pattern" : "replacement"
    If find a NULL - then replace with ''
    '''
    
    replace_excel = pd.read_excel(excel_path)
    
    # Get nulls out
    replace_excel['Replace'][replace_excel['Replace'].isnull()] = ''
    
    # Get dic of replacements - will remove up to the end of the word
    replacements = replace_excel.apply(lambda x: {'\\b' + x['Find'] + '\\b' if '*' not in x['Find'] else x['Find'] : x['Replace']}, axis = 1)
    
    # Then turn into dic 
    return {find: replace for replace_pair in replacements.values.tolist() for find, replace in replace_pair.items()}

In [17]:
replacement_dic = build_replacedic('word_replacement_1.xlsx')
print(replacement_dic)

{'caixa\\S*': '', '\\bcorreu\\b': 'correo', '\\bcanvi\\b': 'cambio', '\\bhola\\b': '', '\\beducaciónfinanciera\\b': 'educación financiera', '\\bvia\\b': '', '\\bet\\b': 'te', '\\bentitat\\b': 'entidad', '\\bdades\\b': 'datos', '\\bcom\\b': 'como', 'gracia\\S*': '', '\\bteva\\b': 'tu', '\\bels\\b': 'los', '\\bbank\\b': '', '\\bcontrasenyes\\b': 'contraseñas', '\\bvía\\b': '', '\\bdomicili\\b': 'domicilio', '\\bmai\\b': 'nunca', '\\bmillones\\b': 'millón', '\\bibex35\\b': 'ibex', '\\bdemanarà\\b': 'demandará', '\\bamb\\b': 'con', '\\bseu\\b': 'su', '\\bbanca\\b': '', 'banco\\S*': '', '\\bpersonals\\b': 'personales', '\\bfem\\b': 'hacemos', '\\bvlcstartupawards\\b': 'startup', '\\bmillon\\b': 'millón', '\\bpolítics\\b': 'políticos', '\\bho\\b': 'lo', 'empresa\\S*': '', '\\brt\\b': '', '\\bdemanen\\b': 'demandan'}


In [18]:
# Now will replace in our data frame and then process the results again
tweet_data['Clean_text'] = tweet_data['Clean_text'].replace(replacement_dic, regex = True)

tweet_list = tweet_data['Clean_text'].values

In [19]:
# POS that we will remove
pos_remove = []

treated_data = basic_treatment(tweet_list, pos_remove)

Have processed 0 tweets
Have processed 500 tweets
Have processed 1000 tweets
Have processed 1500 tweets
Have processed 2000 tweets
Treated data - shape is 2388


In [20]:
for i in range(10):
    print('The original tweet was: \n{}'.format(tweet_list[i]))
    print('The new tweet is: \n{}'.format(treated_data[i]))
    print('\n\n')

The original tweet was: 
perdonar millón  santander  bankia imponer sociedades  crédito
The new tweet is: 
perdonar millón santander bankia imponer sociedad crédito



The original tweet was: 
 financiar año sector hostelero milllones euro
The new tweet is: 
financiar año sector hostelero milllones euro



The original tweet was: 
destinar millón modernización sector turístico hotelero
The new tweet is: 
destinar millón modernización sector turístico hotelero



The original tweet was: 
deber pedir préstamo
The new tweet is: 
deber pedir préstamo



The original tweet was: 
 colocar millón cocos demandar superior  
The new tweet is: 
colocar millón coco demandar superior



The original tweet was: 
 financiar año sector hostelero milllones euro
The new tweet is: 
financiar año sector hostelero milllones euro



The original tweet was: 
 financiar año sector hostelero milllones euro
The new tweet is: 
financiar año sector hostelero milllones euro



The original tweet was: 
 mantener ap

In [21]:
#tweet_data['Clean_text'][tweet_data['Clean_text'].str.contains('mtero')].values
#tweet_data['Clean_text'][tweet_data['Clean_text'].str.contains('tico ')].values
#tweet_data['Clean_text'][tweet_data['Clean_text'].str.contains('der')].values
#tweet_data['Clean_text'][tweet_data['Clean_text'].str.contains('gina ')].values
#tweet_data['Clean_text'][tweet_data['Clean_text'].str.contains('cabk')].values
tweet_data['Clean_text'][tweet_data['Clean_text'].str.contains('ear')].values
#tweet_data['Clean_text'][tweet_data['Clean_text'].str.contains('rea')].values
#tweet_data['Clean_text'][tweet_data['Clean_text'].str.contains('mina')].values
#tweet_data['Clean_text'][tweet_data['Clean_text'].str.contains('test')].values
#tweet_data['Clean_text'][tweet_data['Clean_text'].str.contains('amp')].values
#tweet_data['Clean_text'][tweet_data['Clean_text'].str.contains('gual')].values

array(['abusivo pedir nómina bloquear puto ',
       'oportunidad benidorm convocar xiv concurso ideas emprendedoras crear ',
       ' empezar apostar actividad físico ejercicio mejorar salud rendimiento emplear',
       ' sano emplear corporatewellness',
       ' destinar emplear atenciónalcliente',
       'psoe condonar deuda  crear',
       'actualidad económico china esperar crecer  research traer actualidad económico ent',
       'actualidad económico china esperar crecer  research traer actualidad económico',
       'meu comerç online pots crear teu propi canal vender on line',
       'comercio online poder crear canal venta on line',
       ' mónica  escribirnos desear pue',
       'entender hora competencia abrir emplear bordería',
       'intentar enviar quejar correar electrónico salir distinguido informar',
       ' comprar bitcoins tarjeta credito  coinbase bloquear tarjeta credi',
       'cajero bloquear pantalla blanco número asistencia comunicar soluciones',
       'empl

In [22]:
tweet_data['Clean_text_2'] = treated_data

In [23]:
get_wordcloud(treated_data)

In [24]:
# Check those that are now empty
sum(tweet_data['Clean_text_2'] == '')

119

In [25]:
# Get the most common words
Counter(' '.join(tweet_data['Clean_text_2'].values).split()).most_common(20)

[('financiero', 125),
 ('millón', 108),
 ('premiar', 101),
 ('sabadell', 90),
 ('entidad', 88),
 ('impulsar', 80),
 ('bbva', 71),
 ('colocar', 69),
 ('educación', 66),
 ('programar', 66),
 ('bankia', 61),
 ('funcas', 61),
 ('santander', 59),
 ('google', 56),
 ('emprendedorxxi', 53),
 ('unir', 52),
 ('pagar', 51),
 ('ganar', 51),
 ('año', 50),
 ('director', 49)]

## Bigrams

Now will analyse and join a few bigrams

In [26]:
tweet_bigrams = [list(nltk.bigrams(tweet.split())) for tweet in tweet_data['Clean_text_2'].values]
bigram_list = [bigram for tweet in tweet_bigrams for bigram in tweet]
Counter(bigram_list).most_common(100)

[(('educación', 'financiero'), 62),
 (('programar', 'funcas'), 53),
 (('colocar', 'millón'), 53),
 (('jordi', 'gual'), 41),
 (('unir', 'programar'), 39),
 (('millón', 'coco'), 38),
 (('entidad', 'financiero'), 34),
 (('promover', 'educación'), 33),
 (('comerciar', 'electrónico'), 32),
 (('funcas', 'promover'), 30),
 (('re', 'engagement'), 30),
 (('estrategia', 'app'), 28),
 (('app', 're'), 28),
 (('engagement', 'google'), 28),
 (('impulsar', 'promoción'), 27),
 (('europa', 'utilización'), 26),
 (('utilización', 'estrategia'), 26),
 (('líder', 'europa'), 26),
 (('financiero', 'líder'), 26),
 (('promoción', 'comerciar'), 25),
 (('bbva', 'sabadell'), 24),
 (('electrónico', 'pequeño'), 24),
 (('colaborar', 'impulsar'), 23),
 (('demandar', 'superior'), 23),
 (('premiar', 'emprendedorxxi'), 23),
 (('coco', 'demandar'), 23),
 (('sabadell', 'blackstone'), 22),
 (('blackstone', 'disparar'), 22),
 (('crisis', 'catalán'), 22),
 (('impulsar', 'crecimiento'), 22),
 (('disparar', 'titulizaciones'), 

In [27]:
bigram_replace = {
    'educación financiero' : 'educación_financiero',
    'formación financiero' : 'educación_financiero',
    'entidad financiero' : 'entidad_financiero',
    '(jordi gual|jordigual)' : 'jordi_gual',
    '(emprendedor xxi|emprendedorxxi|xxi)' : 'emprendedor_xxi',
    'comunitat valenc\S*' : 'comunitat_valenciano',
    'crisis catalán' : 'crisis_catalán',
    'tarjeta visar' : 'tarjeta_visa'
}

In [28]:
tweet_data['Clean_text_3'] = tweet_data['Clean_text_2'].replace(bigram_replace, regex = True)

In [29]:
get_wordcloud(tweet_data['Clean_text_3'].values)

## Finall save the data

Save the final data frame to CSV, dropping those that are empty or have only few characters

In [32]:
final_data = tweet_data[tweet_data['Clean_text_3'].str.len() > 3]

print('The cleaned data has shape {}'.format(tweet_data.shape))
print('The final data has shape {}'.format(final_data.shape))

The cleaned data has shape (2388, 6)
The final data has shape (2265, 6)


In [33]:
final_data.to_csv(output_tweets, index = False, encoding = 'utf-8')